# C3 — Atualização de Pose (posição e orientação) em Servo Visual (Difícil)

Estime o delta de pose (posição e orientação) — [rot_z, tx, ty] — a partir de correspondências esparsas de features. Retorne estimativas robustas com ~20% de outliers.

### Contexto
Servo visual alinha a pose (posição e orientação) da câmera do robô a uma vista desejada rastreando correspondências de features entre quadros consecutivos. Os pares de pontos $(X1, X2)$ incluem inliers que obedecem a um pequeno delta de pose em SE(2) e outliers de mismatches/oclusões. Sintetizamos dados via [`common.data_utils.visual_servo_dataset`](common/data_utils.py): cada amostra tem até 20% de outliers mais ruído de sensor, e deltas verdadeiros $[\Delta\theta_z, \Delta t_x, \Delta t_y]$ dentro de ±0.01–0.02 radianos/metros.

### Por que quântico?
Estimação robusta de pose com outliers pode ser formulada como busca combinatória por consenso (variantes de RANSAC). Sub‑rotinas quânticas (por exemplo, amplificação de amplitude para contagem de inliers, QAOA para pontuar subconjuntos) podem acelerar a avaliação de hipóteses ou explorar conjuntos de consenso mais amplos em comparação a heurísticas puramente clássicas.

## Especificação de Entrada e Saída
**Entrada:** `X1 (N×M×2)`, `X2 (N×M×2)`  
**Saída:** `dpose (N×3)` com colunas `[rot, tx, ty]`  
**Tipo de retorno:** array float (`np.float64` recomendado)

Orientações de runtime/recursos: qubits ≤ 12, passos do otimizador ≤ 250; mantenha o tempo por amostra ≤ ~0,2 s.

### Dicas de solução
- Objetivo robusto: minimizar resíduos de Tukey/Huber entre $X1$ transformado e $X2$ ou maximizar o número de inliers sob um limiar de consenso.
- Ângulo quântico: pontuar subconjuntos candidatos via QAOA/Ising (selecionar inliers) ou usar estimação de amplitude para avaliar o suporte da hipótese de forma eficiente.
- Fluxo prático: (1) gerar hipóteses de pose (posição e orientação) — por exemplo, solucionadores mínimos com 2–3 correspondências —, (2) avaliar/impulsionar o consenso com método quântico, (3) refinar a melhor pose com mínimos quadrados ponderados.

_Orientação de baseline:_ [`common.baselines.baseline_visual_servo`](common/baselines.py) obtém medianas de erro ≈ [0.006 rad, 0.012 m, 0.012 m] em seeds variadas; testes públicos exigem medianas de tradução ≤ 0.02.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
import pennylane as qml
np.random.seed(1337)

## Baseline (referência)

In [2]:
X1, X2, y = du.visual_servo_dataset(n=64, m=24, seed=0, outlier_rate=0.2)
pred = bl.baseline_visual_servo(X1, X2)
err = np.median(np.abs(pred - y), axis=0)
print("Mediana do |erro| do baseline [rot, tx, ty]:", err)

Mediana do |erro| do baseline [rot, tx, ty]: [0.0122711  0.00318421 0.00262652]


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(X1, X2):
    
    # --- Início da Solução C3 ---
    
    # 1. Imports
    import pennylane as qml
    from pennylane import numpy as pnp
    from common import quantum_utils as qu
    from common import baselines as bl 

    # --- 2. Funções Clássicas (Diferenciáveis) ---
    def transform(x1_points, pose):
        d_rot, d_tx, d_ty = pose[0], pose[1], pose[2]
        R = pnp.array([
            [pnp.cos(d_rot), -pnp.sin(d_rot)],
            [pnp.sin(d_rot), pnp.cos(d_rot)]
        ])
        T = pnp.array([d_tx, d_ty])
        x1_transformed = (pnp.dot(R, x1_points.T)).T + T
        return x1_transformed

    def robust_huber_cost(pose, x1, x2, delta):
        x1_transformed = transform(x1, pose)
        errors = pnp.linalg.norm(x1_transformed - x2, axis=1)
        is_inlier = errors <= delta
        is_outlier = ~is_inlier
        inlier_loss = pnp.sum(0.5 * (errors[is_inlier]**2))
        outlier_loss = pnp.sum(delta * (errors[is_outlier] - 0.5 * delta))
        return inlier_loss + outlier_loss

    # --- 3. Circuito Quântico (Propositor de Delta) ---
    n_qubits = 3 
    n_layers = 2
    dev = qu.default_device(n_qubits=n_qubits)

    @qml.qnode(dev)
    def quantum_delta_proposer(params):
        qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
        return [qml.expval(qml.Z(i)) for i in range(n_qubits)]

    # --- 4. Função de Custo Híbrida ---
    def hybrid_cost_fn(q_params, x1_sample, x2_sample, pose_baseline, huber_delta, scale):
        exp_values = quantum_delta_proposer(q_params)
        delta_pose = pnp.array(exp_values) * scale
        pose_proposed = pose_baseline + delta_pose
        return robust_huber_cost(pose_proposed, x1_sample, x2_sample, huber_delta)

    # --- 5. Otimização Principal ---
    N = len(X1)
    dposes_out = pnp.zeros((N, 3))
    
    # Parâmetros de otimização
    steps = 8 
    lr = 0.020
    huber_delta = 0.02
    delta_scale = pnp.array([0.01, 0.02, 0.02]) 
    
    opt = qml.AdamOptimizer(stepsize=lr)
    
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_qubits)
    
    dpose_baselines = bl.baseline_visual_servo(X1, X2)

    for i in range(N):
        x1_i = pnp.array(X1[i])
        x2_i = pnp.array(X2[i])
        pose_baseline_i = pnp.array(dpose_baselines[i])
        q_params = pnp.random.normal(0, 0.01, size=params_shape, requires_grad=True)
        
        for _ in range(steps):
            q_params = opt.step(
                lambda v: hybrid_cost_fn(v, x1_i, x2_i, pose_baseline_i, huber_delta, delta_scale), 
                q_params
            )

        # --- 6. Obter Resultado Final ---
        final_exp_values = quantum_delta_proposer(q_params)
        final_delta = pnp.array(final_exp_values) * delta_scale
        final_pose = pose_baseline_i + final_delta
        
        dposes_out[i, :] = final_pose
            
    return dposes_out.unwrap()

# --- Fim da Solução C3 ---

## Testes públicos (rápidos)

In [12]:
X1, X2, y = du.visual_servo_dataset(n=64, m=24, seed=1, outlier_rate=0.2)
pred = solve(X1, X2)
med = np.median(np.abs(pred - y), axis=0)
print("Mediana pública |erro|:", med)
assert pred.shape == y.shape
assert med[1] <= 0.02 and med[2] <= 0.02, "Erro mediano de tradução muito alto"
print("OK")

Mediana pública |erro|: [0.01107799 0.01895527 0.01959526]
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.